In [1]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers, models

import time
import os
import matplotlib.pyplot as plt

import numpy as np

import pickle
import gzip

2024-10-31 05:10:45.591515: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-31 05:10:48.284233: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
import os
os.getcwd()

In [2]:
with gzip.open('../../result/one_hot_encoding/gene_id_label_ohe.pkl.gz', 'rb') as f:
    data = pickle.load(f)

In [3]:
data

,ensembl_gene_id,DE,upstream_region_encoded
0,ENSG00000000457,1,"[[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
1,ENSG00000000460,0,"[[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 0, 1], [0,..."
2,ENSG00000000938,0,"[[1, 0, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [1,..."
3,ENSG00000000971,1,"[[0, 1, 0, 0], [0, 0, 1, 0], [0, 1, 0, 0], [0,..."
4,ENSG00000001460,0,"[[0, 0, 0, 1], [0, 0, 0, 1], [1, 0, 0, 0], [0,..."
...,...,...,...
55221,ENSG00000284520,0,"[[0, 0, 0, 1], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."
55222,ENSG00000284544,0,"[[0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 0, 1], [1,..."
55223,ENSG00000284554,0,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 1, 0], [1,..."
55224,ENSG00000284568,0,"[[1, 0, 0, 0], [0, 0, 0, 1], [1, 0, 0, 0], [0,..."


## Quality Control

In [ ]:
data.loc[0, "upstream_region_encoded"].shape

In [ ]:
accum = 0

for i in range(len(data)):
    temp  = test.iloc[1][['upstream_region_encoded']]
    if temp["upstream_region_encoded"].shape != (2001, 4):
        accum += 1

    if accum == 0:
        print('All genes have the same dimension of one-hot-encoded upstream region.')
print(accum)

## Train-test Split

In [6]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=123)

In [7]:
train_data

,ensembl_gene_id,DE,upstream_region_encoded
20681,ENSG00000218672,0,"[[1, 0, 0, 0], [0, 0, 1, 0], [1, 0, 0, 0], [0,..."
46371,ENSG00000026559,1,"[[0, 0, 1, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,..."
7859,ENSG00000234415,0,"[[0, 0, 1, 0], [0, 1, 0, 0], [0, 1, 0, 0], [0,..."
12713,ENSG00000249441,0,"[[1, 0, 0, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0,..."
32644,ENSG00000089250,0,"[[0, 1, 0, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
...,...,...,...
54370,ENSG00000268461,0,"[[1, 0, 0, 0], [1, 0, 0, 0], [0, 1, 0, 0], [0,..."
17730,ENSG00000253015,0,"[[0, 0, 1, 0], [1, 0, 0, 0], [1, 0, 0, 0], [0,..."
28030,ENSG00000110375,0,"[[0, 0, 1, 0], [1, 0, 0, 0], [0, 0, 0, 1], [0,..."
15725,ENSG00000213500,0,"[[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0,..."


In [8]:
BUFFER_SIZE = 50000
BATCH_SIZE = 256

In [9]:
train_data = tf.data.Dataset.from_tensor_slices(train_data)
train_data_shuffled = train_data.shuffle(BUFFER_SIZE)

2024-10-30 21:11:46.770342: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-30 21:11:46.771332: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-30 21:11:46.772191: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

InternalError: cudaGetDevice() failed. Status: CUDA driver version is insufficient for CUDA runtime version

In [ ]:

class SequenceDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, dataframe, batch_size=128, sequence_column="upstream_region_encoded"):
        self.dataframe = dataframe
        self.batch_size = batch_size
        self.sequence_column = sequence_column
        self.indices = np.arange(len(dataframe))
    
    def __len__(self):
        # Number of batches per epoch
        return int(np.floor(len(self.dataframe) / self.batch_size))

    def __getitem__(self, index):
        # Generate batch indices
        batch_indices = self.indices[index * self.batch_size:(index + 1) * self.batch_size]
        
        # Load and stack sequences for the batch
        batch_sequences = np.stack(self.dataframe.iloc[batch_indices][self.sequence_column].values)

        # Transpose each sequence to channels-last format
        batch_sequences = np.transpose(batch_sequences, (1, 2, 0)) 
        
        return batch_sequences

# Initialize generator
batch_size = 32
sequence_generator = SequenceDataGenerator(train_data, batch_size=batch_size)

# Example: Access the first batch
first_batch = sequence_generator[0]
print("First batch shape:", first_batch.shape)

In [ ]:
def build_generator(sequence_length=2001, depth=4, batch_size=128, noise_dim=100):
    model = tf.keras.Sequential()
    
    # Start with a dense layer, which we'll reshape to the target shape
    model.add(layers.Dense(sequence_length * depth * batch_size, input_dim=noise_dim, activation='relu'))
    model.add(layers.Reshape((sequence_length, depth, batch_size)))
    
    # Add additional layers if needed to increase complexity
    model.add(layers.Conv2DTranspose(64, (5, 1), strides=(2, 1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2DTranspose(32, (5, 1), strides=(2, 1), padding='same', activation='relu'))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2DTranspose(depth, (5, 1), padding='same', activation='softmax'))  # Output depth channels

    return model

In [ ]:
generator = build_generator()
generator.summary()

In [ ]:
def build_generator(sequence_length):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=100, activation='relu'))
    model.add(layers.Dense(sequence_length * 4, activation='softmax'))  # output should match one-hot size
    model.add(layers.Reshape((sequence_length, 4)))  # Reshape to (sequence_length, 4) for one-hot
    return model

In [ ]:
# one_hot_encoded = np.array([
#     [1, 0, 0, 0],  # A
#     [0, 1, 0, 0],  # T
#     [0, 0, 1, 0],  # C
#     [0, 0, 0, 1],  # G
#     [1, 0, 0, 0]   # A
# ])

# # Mapping from index to nucleotide
# index_to_nucleotide = ['A', 'T', 'C', 'G']

# # Convert one-hot encoding back to sequence
# data['sequence'] = data['upstream_region_encoded'].apply(
#     lambda one_hot_encoded: ''.join([index_to_nucleotide[np.argmax(row)] for row in one_hot_encoded])
# )

In [ ]:
data

In [ ]:
# data = data[['ensembl_gene_id', 'sequence']]

In [ ]:
data

In [3]:
len(tf.config.list_physical_devices('GPU'))

2024-10-31 05:11:25.540359: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-10-31 05:11:25.541804: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


0

In [2]:
tf.__version__

'2.16.1'